In [2]:
import pandas as pd
import yaml

In [180]:
class node:

    def __init__(self):
        self.up = self
        self.down = self
        self.left = self
        self.right = self
        self.COL = None
        self.ROW = None

    def hideInCol(self):
        self.up.down = self.down
        self.down.up = self.up
        self.COL.row_count -= 1
        return self

    def showInCol(self):
        self.up.down = self
        self.down.up = self
        self.COL.row_count += 1
        return self

    def hideInRow(self):
        self.left.right = self.right
        self.right.left = self.left
        return self

    def showInRow(self):
        self.left.right = self
        self.right.left = self
        return self
    
    def deleteUnuseableRow(self):
        cur_cell = self.right 
        while cur_cell != self:
            print(cur_cell.COL.label)
            cur_cell.hideInCol()
            cur_cell = cur_cell.right
        print("done")
        self.hideInCol()
        return

    def hideWholeRow(self, start = None):
        if start == None: # if we just started the loop, don't hide me from my column, but go to my neighbor
            self.right.hideWholeRow(start = self)
            return
        if start != self: # we are not first and not last => hide me and continue
            self.hideInCol()
            self.right.hideWholeRow(start = start)
            return
        return # last possibility start == self => we have looped and can finish

    def showWholeRow(self, start = None):
        if start == None: # if we just started the loop, don't hide me from my column, but go to my neighbor
            self.left.showWholeRow(start = self)
            return
        if start != self: # we are not first and not last => hide me and continue
            self.showInCol()
            self.left.showWholeRow(start = start)
            return
        return # last possibility start == self => we have looped and can finish
    
    def getNeighbor(self, direction, by=0):
        neighbor = {
            'left'  : self.left,
            'right' : self.right,
            'up'    : self.up,
            'down'  : self.down
        }[direction]
        if by > 0:
            return neighbor.getNeighbor(direction, by = by-1)
        return neighbor




In [183]:
class columnHeader(node):

    def __init__(self, label):
        super().__init__()
        self.label = label
        self.row_count = 0
        self.COL = self

    def hideMe(self):
        self.hideInRow()
        cell = self.down
        while cell != self:
            cell.hideWholeRow()
            cell = cell.down
        return self

    def showMe(self):
        self.showInRow()
        cell = self.up
        while cell != self:
            cell.showWholeRow()
            cell = cell.up
        return self

    def appendRow(self, row):
        row.down = self
        row.up = self.up
        self.up.down = row
        self.up = row
        row.COL = self
        self.row_count += 1
        return self

    def addToRoot(self, root):
        root.appendColumn(self)
        return self

In [181]:
class rowHeader(node):
    
    def __init__(self, label):
        super().__init__()
        self.label = label
        self.ROW = self

    def appendColumn(self, col):
        col.right = self
        col.left = self.left
        self.left.right = col
        self.left = col
        col.ROW = self
        return self

    def addToRoot(self, root):
        root.appendRow(self)
        return self


In [184]:
class rootNode(columnHeader, rowHeader):

    def __init__(self):
        node.__init__(self)
        self.row_count = 0
        self.label = None
        self.ROW = self
        self.COL = self

    def addToRoot(self, root):
        pass

In [307]:
def append_dict(d, e):
    r = d.copy()
    r.update(e)
    return r

In [164]:
def get_dimensions_form_riddle (r_t):
    # note : there are two main types of entnries
    # - entires that say "X is alo Y" which are represneted by dict with two entires
    #   [ 
    #     X_category: X_value,
    #     Y_category: Y_value
    #   ]
    # - special entires represneted by a dict. wiht only one entru
    #  ( the key is then one of: _adj, _left, _right, _add)
    # [item for item in riddle_text if len(item) != 1]

    basic_items = [
        {dim : val}
            for item in r_t if len(item) !=1
                for dim, val in item.items() if dim != '_#'
    ]

    additional_items = [
        dim_val
            for special_item in r_t if len(special_item) == 1
                for attributes in special_item.items() 
                    for attribute in attributes if type(attribute) == list
                        for dim_val in attribute
    ]

    all_items = basic_items + additional_items
    dictionaries = {dim : set() for dim_val in all_items for dim in dim_val}
    {dictionaries[dim].add(val) for item in all_items for dim, val in item.items()}
    
    max_items = max([len(v) for k, v in dicts.items()])
    dictionaries['_#'] = {r+1 for r in range(max_items)}

    return dictionaries

In [371]:
def check_simple_criterion(sparse_matrix_row_attr, check_attr):
    
    has_dims = {k for k in sparse_matrix_row_attr}
    needs_dims = {k for k in check_attr}
    overlap_attrs = has_dims & needs_dims
    can_be_checked = len(overlap_attrs) == len(check_attr)

    # not enough attributes present to perform the  check => default to OK
    if not can_be_checked:
        return True

    # check on each dimension OK => all OK
    summary_check_result = { sparse_matrix_row_attr[d] == check_attr[d] for d in overlap_attrs } 

    return summary_check_result == {True} or summary_check_result == {False}



In [372]:
def check_set_of_criteria(sparse_matrix_row_attr, check_set):

    summary_check_results = {
        check_simple_criterion(sparse_matrix_row_attr, item)
        for item in check_set
    }
    
    return summary_check_results == {True}

In [77]:
riddle_file = "input_1_einstein.yml"

In [113]:
with open(riddle_file) as f:
    try:
        riddle_text= yaml.safe_load(f)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
simple_guards = [item for item in riddle_text if len(item) !=1] 

[{'nation': 'Norway', '_#': 1},
 {'nation': 'England', 'house_color': 'red'},
 {'nation': 'Dutch', 'drink': 'tea'},
 {'cig': 'cigars', 'house_color': 'yellow'},
 {'nation': 'Germany', 'cig': 'pipe'},
 {'_#': 3, 'drink': 'milk'},
 {'cig': 'no_filter', 'pet': 'birds'},
 {'nation': 'Sweden', 'pet': 'dogs'},
 {'cig': 'menthol', 'drink': 'beer'},
 {'house_color': 'green', 'drink': 'coffee'}]

In [328]:
dicts = get_dimensions_form_riddle(riddle_text)


In [261]:
dicts

{'nation': {'Dutch', 'England', 'Germany', 'Norway', 'Sweden'},
 'house_color': {'blue', 'green', 'red', 'white', 'yellow'},
 'drink': {'beer', 'coffee', 'milk', 'tea', 'water'},
 'cig': {'cigars', 'light', 'menthol', 'no_filter', 'pipe'},
 'pet': {'birds', 'cat', 'dogs', 'fish', 'horses'},
 '_#': {1, 2, 3, 4, 5}}

In [262]:
simple_guards

[{'nation': 'Norway', '_#': 1},
 {'nation': 'England', 'house_color': 'red'},
 {'nation': 'Dutch', 'drink': 'tea'},
 {'cig': 'cigars', 'house_color': 'yellow'},
 {'nation': 'Germany', 'cig': 'pipe'},
 {'_#': 3, 'drink': 'milk'},
 {'cig': 'no_filter', 'pet': 'birds'},
 {'nation': 'Sweden', 'pet': 'dogs'},
 {'cig': 'menthol', 'drink': 'beer'},
 {'house_color': 'green', 'drink': 'coffee'}]

In [431]:
class einstein_matrix:

    def __init__(self):

        self.matrix_root = rootNode()
        self.column_headers = {}
        self.row_headers = []

    def create_columns(self, dicts):

        self.column_headers = {
            dim :{val : columnHeader(f"{dim}:{val}").addToRoot(self.matrix_root) for val in vals} for dim, vals in dicts.items()
        }

        return self

    def create_row_labels(self, dicts, guards):
    
        level = 0
        labels = [{}]
        copy_dics = dicts.copy()

        while len(copy_dics) > 0:
            dim = copy_dics.popitem()
            dim_name = dim[0]
            dim_values = dim[1]
            labels = [
                append_dict(label, {dim_name: value})
                for label in labels 
                for value in dim_values
                if check_set_of_criteria(
                    append_dict(label, {dim_name: value}),
                guards)
            ]

        return labels

    def create_rows(self, dicts, guards):
        
        labels = self.create_row_labels(dicts, guards)

        self.row_headers = [
            {
                "attrs" : l,
                "row_header" :
                    rowHeader(
                        ":".join(
                            [
                                str(i) for i in list(l.values())
                            ]
                        )
                    ).addToRoot(self.matrix_root)
                
            }   for l in labels
        ]

        return self





In [432]:
em = einstein_matrix()
em.create_columns(dicts)
em.create_rows(dicts, simple_guards)

In [ ]:
def search(matrix, solution=[], log=[], k=0):

    # print(f"k : {k}")

    if type(matrix) != rootNode:
        raise TypeError("Must start with matrix root node")

    # solution found if matrix is empty
    if matrix.right == matrix:
        print("ready")
        return prep_solution(solution)

    # select column
    col = matrix.right
    tmp_col = col
    while tmp_col != matrix:
        if tmp_col.row_count < col.row_count:
            col = tmp_col
        tmp_col = tmp_col.right
    print(col.label)
    # hide column 
    col.hideMe()
    # go through its each row
    row = col.down
    print(row.ROW.label)
    while row != col:
        # add row to solution
        solution.append(row)
        log.append(f"select R{row.ROW.label['R']}C{row.ROW.label['C']}={row.ROW.label['#']}")
        # hide row's columns
        cell = row.right
        while cell != row:
            if type(cell) != rowHeader:
                cell.COL.hideMe()
            cell = cell.right
        # go deeper
        result = search(matrix, solution, log, k = k+1)
        if result != None:
            return result
        # uncover columns and remove row from solution
        solution.pop()
        log.append("back")
        cell = row.left
        while cell != row:
            if type(cell) != rowHeader:
                cell.COL.showMe()
            cell = cell.left
        # next row
        row = row.down
    # # uncover column
    col.showMe()
    return None

    